# Grupowanie metodą k-średnich

k-średnich jest metodą:

- partycjonującą (granice każdego skupienia są niezależne od pozostałych)
- na wyłączność (każdy element może należeć tylko do jednego skupienia)
- kompletną (każdy element w zbiorze danych musi być przypisany do przynajmniej jednego skupienia).

## Zadania

1.	Wczytaj plik `college.csv`, a następnie podejrzyj zawartość ramki danych. Zbiór zawiera informacje o 1270 szkołach wyższych i uniwersytetach w Stanach Zjednoczonych.

In [7]:
library(tidyverse)
options(jupyter.rich_display = FALSE,
        repr.plot.width = 15,
        repr.plot.height = 8)

In [8]:
df <- read.csv("data/college.csv")

In [9]:
head(df)

  id     name                                city       state region
1 102669 Alaska Pacific University           Anchorage  AK    West  
2 101648 Marion Military Institute           Marion     AL    South 
3 100830 Auburn University at Montgomery     Montgomery AL    South 
4 101879 University of North Alabama         Florence   AL    South 
5 100858 Auburn University                   Auburn     AL    South 
6 100663 University of Alabama at Birmingham Birmingham AL    South 
  highest_degree control gender admission_rate sat_avg undergrads tuition
1 Graduate       Private CoEd   0.4207         1054      275      19610  
2 Associate      Public  CoEd   0.6139         1055      433       8778  
3 Graduate       Public  CoEd   0.8017         1009     4304       9080  
4 Graduate       Public  CoEd   0.6788         1029     5485       7412  
5 Graduate       Public  CoEd   0.8347         1215    20514      10200  
6 Graduate       Public  CoEd   0.8569         1107    11383       7510  

In [10]:
names(df)

 [1] "id"                 "name"               "city"              
 [4] "state"              "region"             "highest_degree"    
 [7] "control"            "gender"             "admission_rate"    
[10] "sat_avg"            "undergrads"         "tuition"           
[13] "faculty_salary_avg" "loan_default_rate"  "median_debt"       
[16] "lon"                "lat"               

In [11]:
dim(df)

[1] 1270   17

In [12]:
glimpse(df)

Rows: 1,270
Columns: 17
$ id                 <int> 102669, 101648, 100830, 101879, 100858, 100663, 101…
$ name               <chr> "Alaska Pacific University", "Marion Military Insti…
$ city               <chr> "Anchorage", "Marion", "Montgomery", "Florence", "A…
$ state              <chr> "AK", "AL", "AL", "AL", "AL", "AL", "AL", "AL", "AL…
$ region             <chr> "West", "South", "South", "South", "South", "South"…
$ highest_degree     <chr> "Graduate", "Associate", "Graduate", "Graduate", "G…
$ control            <chr> "Private", "Public", "Public", "Public", "Public", …
$ gender             <chr> "CoEd", "CoEd", "CoEd", "CoEd", "CoEd", "CoEd", "Co…
$ admission_rate     <dbl> 0.4207, 0.6139, 0.8017, 0.6788, 0.8347, 0.8569, 0.8…
$ sat_avg            <int> 1054, 1055, 1009, 1029, 1215, 1107, 1041, 1165, 107…
$ undergrads         <int> 275, 433, 4304, 5485, 20514, 11383, 7060, 3033, 264…
$ tuition            <int> 19610, 8778, 9080, 7412, 10200, 7510, 7092, 27324, …
$ faculty_salary

In [ ]:
# df <- df %>%
#   mutate(
#     Trailer_views = as.integer(Trailer_views),
#     is_3D_available = is_3D_available == "YES",
#     Genre = as.factor(Genre),
#     Avg_age_actors = as.integer(Avg_age_actors),
#     Num_multiplex = as.integer(Num_multiplex),
#     Collection = as.integer(Collection),
#     Start_Tech_Oscar = as.factor(Start_Tech_Oscar),
#   )

2. Utwórz nową ramkę danych, w której zamieścisz dane wyłącznie dla stanu Maryland (`state == MD`). Przekonwertuj nazwy każdej uczelni w etykiety wierszy (funkcja `column_to_rownames` - etykiety są potrzebne przy wizualizacji skupień). Następnie wyświetl statystyki podsumowujące dla cech `admission_rate` (odsetek przyjmowanych kandydatów) oraz `sat_avg` (średnia wyników egzaminów SAT).

In [37]:
df_md <- df %>%
    filter(state == "MD") %>%
    column_to_rownames("name")

In [44]:
summary(df_md[c("admission_rate", "sat_avg")])

 admission_rate      sat_avg    
 Min.   :0.1608   Min.   : 842  
 1st Qu.:0.5181   1st Qu.: 900  
 Median :0.5961   Median :1048  
 Mean   :0.5886   Mean   :1062  
 3rd Qu.:0.6606   3rd Qu.:1176  
 Max.   :0.8696   Max.   :1439  



3. Znormalizuj dane dla wybranych w punkcie 2 cech metodą z-score (funkcja `scale()`). Następnie dla nowo utworzonego zbioru wyświetl i przeanalizuj statystyki podsumowujące.

In [49]:
df_md_num <- df_md %>%
    select(where(is.numeric))

In [ ]:
names(df_md_num)

[1] "id"                 "admission_rate"     "sat_avg"           
[4] "undergrads"         "tuition"            "faculty_salary_avg"
[7] "median_debt"        "lon"                "lat"               

In [53]:
df_md_num_scaled <- scale(df_md_num)

In [54]:
summary(df_md_num_scaled)

       id          admission_rate        sat_avg          undergrads      
 Min.   :-1.6452   Min.   :-2.77601   Min.   :-1.2512   Min.   :-0.72851  
 1st Qu.:-0.9285   1st Qu.:-0.45725   1st Qu.:-0.9218   1st Qu.:-0.54251  
 Median : 0.1731   Median : 0.04895   Median :-0.0813   Median :-0.27779  
 Mean   : 0.0000   Mean   : 0.00000   Mean   : 0.0000   Mean   : 0.00000  
 3rd Qu.: 0.7485   3rd Qu.: 0.46753   3rd Qu.: 0.6485   3rd Qu.: 0.01344  
 Max.   : 1.4198   Max.   : 1.82387   Max.   : 2.1393   Max.   : 3.15142  
    tuition        faculty_salary_avg  median_debt           lon         
 Min.   :-0.9668   Min.   :-1.6626    Min.   :-1.9207   Min.   :-3.5130  
 1st Qu.:-0.8762   1st Qu.:-0.5987    1st Qu.:-0.7640   1st Qu.:-0.1665  
 Median :-0.4924   Median :-0.3455    Median :-0.1784   Median : 0.2952  
 Mean   : 0.0000   Mean   : 0.0000    Mean   : 0.0000   Mean   : 0.0000  
 3rd Qu.: 0.8710   3rd Qu.: 0.2089    3rd Qu.: 0.8541   3rd Qu.: 0.2952  
 Max.   : 1.7415   Max.   : 2.5

4. Pogrupuj dane metodą k-średnich (funkcja `kmeans()`. Pierwszy argument: dane, drugi - `centers` - liczba skupień, argument `nstart` - liczba konfiguracji do wypróbowania). Ustaw wartości `centers` na 3, zaś `nstart` na 25.

In [56]:
kmeans_model <- kmeans(df_md_num_scaled, centers = 3, nstart = 25)

5. Atrybut `size` funkcji `kmeans` reprezentuje liczbę obserwacji w każdej grupie, natomiast atrybut `centers` podaje współrzędne centroid każdego skupienia. Ile obserwacji znajduje się w każdej z trzech grup i jakie są współrzędne ich centroid?

In [57]:
kmeans_model$size

[1] 9 6 4

In [58]:
kmeans_model$centers

  id         admission_rate sat_avg      undergrads tuition   
1  0.7132741  0.5161299      0.007041032 -0.4246312  0.4634003
2 -1.2658659 -0.2053411     -0.767555475 -0.3814482 -0.5882821
3  0.2939322 -0.8532805      1.135490891  1.5275923 -0.1602274
  faculty_salary_avg median_debt lon        lat       
1 -0.2817820          0.7325281   0.2985534 -0.1642431
2 -0.3176873         -0.2998342  -0.5582510  0.1255351
3  1.1105404         -1.1984369   0.1656313  0.1812443

6. Zwizualizuj skupienia za pomocą funkcji `fviz_cluster()` z pakietu `factoextra`. Funkcja przyjmuje argumenty: wynik grupowania, dane, które posłużyły do utworzenia grup oraz `repel`, który pomaga w zorganizowaniu układu etykiet elementów w wizualizacji. Opisz jakie różnice między skupieniami można zaobserwować na podstawie wykresu.

In [59]:
library(factoextra)


ERROR: Error in library(factoextra): there is no package called ‘factoextra’


In [ ]:
install.packages("factoextra")

also installing the dependencies ‘Deriv’, ‘microbenchmark’, ‘doBy’, ‘SparseM’, ‘MatrixModels’, ‘minqa’, ‘nloptr’, ‘carData’, ‘Formula’, ‘pbkrtest’, ‘quantreg’, ‘lme4’, ‘crosstalk’, ‘estimability’, ‘mvtnorm’, ‘corrplot’, ‘viridis’, ‘car’, ‘DT’, ‘ellipse’, ‘emmeans’, ‘flashClust’, ‘leaps’, ‘multcompView’, ‘scatterplot3d’, ‘ggsci’, ‘cowplot’, ‘ggsignif’, ‘gridExtra’, ‘polynom’, ‘rstatix’, ‘abind’, ‘dendextend’, ‘FactoMineR’, ‘ggpubr’, ‘ggrepel’


Warning message in install.packages("factoextra"):
“installation of package ‘nloptr’ had non-zero exit status”


7. Oceń różnice między skupieniami w zakresie pozostałych atrybutów: ilość osób na studiach licencjackich (`undergrads`), czesne (`tuition`), spłacalność kredytu studenckiego (`loan_default_rate`), pensje pracowników wydziału (`faculty_salary_avg`) czy mediana kredytu absolwentów (`median_debt`). W tym celu należy najpierw przypisać etykiety grup do obserwacji w zbiorze. Następnie wybiera się atrybuty do porównania, grupując je według skupień i generuje średnie wartości dla każdego z wybranych atrybutów.

### Wybieranie odpowiedniej liczby skupień.

D określenia optymalnej liczby skupień można wykorzystać funkcję `fviz_nbclust()` z pakietu `factoextra`. Jako trzeci argument (po zbiorze danych i metodzie grupowania) podaje się metodę oceniania:

- `wss` -  na podstawie miary podobieństwa/ braku podobieństwa elementów w skupieniu WCSS (*Within Cluster Sum of Squares*) - suma kwadratów wewnątrz skupienia, czyli suma odległości między elementami w skupieniu i jego centroidą. Im mniejsza wartość WCSS, tym bardziej elementy w skupieniu są podobne do siebie. Wartość WCSS jest odwrotnie proporcjonalna do liczby `k`. Odpowiednią wartość `k` dobiera się na podstawie krzywej zależności liczby skupień od WCSS - jest to punkt zagięcia, poniżej którego dalszy wzrost `k` nie powoduje już istotnej zmiany WCSS (metoda "łokcia").

- `silhouette` - metoda średniego zarysu - zarys elementu mierzy, jak bardzo element pasuje do innych elementów w tym samym skupieniu i jak mało pasuje do elementów w sąsiednich. Wartość zarysu bliska `1` oznacza, że element znajduje się w odpowiednim skupieniu, wartość `-1` - że znajduje się w nieodpowiednim skupieniu. Jeśli większość elementów ma wysoką wartość, konfiguracja zostaje uznana za odpowiednią.

- `gap_stat` - statystyka odstępu - porównuje różnice między skupieniami tworzonymi na podstawie danych obserwacji i skupieniami tworzonymi z losowo generowanego zbioru danych, nazywanego referencyjnym zbiorem danych. Dla danej wartości *k* statystyka odstępu jest różnicą w całkowitych sumach WCSS danych obserwacji i referencyjnego zbioru danych. Optymalna liczba skupień jest oznaczona przez wartość *k*, która prowadzi do uzyskania najwyższej statystyki odstępu.

8. Na podstawie wspomnianych metod dobierz odpowiednią liczbę skupień, a następnie przeprowadź i zwizualizuj grupowanie k-średnich dla wybranego *k*.